# Routing

Routing allows you to define routes between component ports


In [ ]:
import pp

In [ ]:
c = pp.Component('sample_connect_strip')
mmi1 = c << pp.c.mmi1x2()
mmi2 = c << pp.c.mmi1x2()
mmi2.move((100, 50))
pp.qp(c)

# Connect strip

Connect strip connects using Manhattan routing

In [ ]:
r = pp.routing.connect_strip(mmi1.ports['E1'], mmi2.ports['W0'])

In [ ]:
c.add(r)

In [ ]:
pp.qp(c)

# Connect strip way_points

You can also specify the points along the route

In [ ]:
c = pp.Component('sample_connect_strip_way_points')
mmi1 = c << pp.c.mmi1x2()
mmi2 = c << pp.c.mmi1x2()
mmi2.move((100, 50))

x0 = mmi1.ports['E0'].x
y0 = mmi1.ports['E0'].y


x2 = mmi2.ports['E0'].x
y2 = mmi2.ports['E0'].y

r2 = pp.routing.connect_strip_way_points([(x0, y0),  (x2+40, y0), (x2+40, y2), (x2, y2)])
c.add(r2)
pp.qp(c)
pp.show(c)

You can avoid expanding to wider waveguides by passing a `None` taper factory

In [ ]:
c = pp.Component('sample_connect_strip_way_points_no_tapers')
mmi1 = c << pp.c.mmi1x2()
mmi2 = c << pp.c.mmi1x2()
mmi2.move((100, 50))

x0 = mmi1.ports['E0'].x
y0 = mmi1.ports['E0'].y


x2 = mmi2.ports['E0'].x
y2 = mmi2.ports['E0'].y

r2 = pp.routing.connect_strip_way_points([(x0, y0),  (x2+40, y0), (x2+40, y2), (x2, y2)], taper_factory=None)
c.add(r2)
pp.qp(c)
pp.show(c)

# Connect bundle

A river routing connects many ports to many ports and routes them in a river fashion

In [ ]:
import numpy as np
import pp
from pp import LAYER
from pp import Port
from pp.routing.connect_component import add_io_optical


@pp.autoname
def big_device(w=400.0, h=400.0, N=16, port_pitch=15.0, layer=LAYER.WG, wg_width=0.5):
    """ big device with N ports on each side """
    component = pp.Component()
    p0 = np.array((0, 0))
    dx = w / 2
    dy = h / 2

    points = [[dx, dy], [dx, -dy], [-dx, -dy], [-dx, dy]]
    component.add_polygon(points, layer=layer)
    port_params = {"layer": layer, "width": wg_width}
    for i in range(N):
        port = Port(
            name="W{}".format(i),
            midpoint=p0 + (-dx, (i - N / 2) * port_pitch),
            orientation=180,
            **port_params
        )
        component.add_port(port)

    for i in range(N):
        port = Port(
            name="E{}".format(i),
            midpoint=p0 + (dx, (i - N / 2) * port_pitch),
            orientation=0,
            **port_params
        )
        component.add_port(port)

    for i in range(N):
        port = Port(
            name="N{}".format(i),
            midpoint=p0 + ((i - N / 2) * port_pitch, dy),
            orientation=90,
            **port_params
        )
        component.add_port(port)

    for i in range(N):
        port = Port(
            name="S{}".format(i),
            midpoint=p0 + ((i - N / 2) * port_pitch, -dy),
            orientation=-90,
            **port_params
        )
        component.add_port(port)
    return component


In [ ]:
c = pp.Component('sample_connect_bundle')
c1 = c << big_device(N=20)
c2 = c << big_device(N=20)
c2.move((600, 600))

c1_east_ports = [v for k, v in c1.ports.items() if k[0] == 'E'] # east facing ports
c2_west_ports = [v for k, v in c2.ports.items() if k[0] == 'W'] # west facing ports

routes = pp.routing.connect_bundle(start_ports=c1_east_ports, end_ports=c2_west_ports)
c.add(routes)
pp.qp(c)
pp.show(c)

You can also increase the separation between the routes of the river_router

In [ ]:
c = pp.Component('sample_connect_bundle_larger_separation')
c1 = c << big_device(N=20)
c2 = c << big_device(N=20)
c2.move((900, 900))

c1_east_ports = [v for k, v in c1.ports.items() if k[0] == 'E'] # east facing ports
c2_west_ports = [v for k, v in c2.ports.items() if k[0] == 'W'] # west facing ports

routes = pp.routing.connect_bundle(start_ports=c1_east_ports, end_ports=c2_west_ports, separation=15)
c.add(routes)
pp.qp(c)
pp.show(c)